In [ ]:
import os
import shutil
from datetime import datetime
import sys
import json
import argparse

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

In [ ]:
PROJECT = "ml-practice-260405"
BUCKET = "bucket-ml-practice-260405"
REGION = "us-central1"
MODEL_TYPE = "cnn"  # "linear", "dnn", "dnn_dropout", or "cnn"

In [ ]:
# Do not change 
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "2.1"  # Tensorflow version

## Input functions to read JPEG images

The key difference between this notebook and [the MNIST one](./mnist_models.ipynb) is in the input function.
In the input function here, we are doing the following:
* Reading JPEG images, rather than 2D integer arrays.
* Reading in batches of batch_size images rather than slicing our in-memory structure to be batch_size images.
* Resizing the images to the expected HEIGHT, WIDTH. Because this is a real-world dataset, the images are of different sizes. We need to preprocess the data to, at the very least, resize them to constant size.

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [ ]:
%%bash
mkdir flowersmodel_augment_tf_v_2_1
mkdir flowersmodel_augment_tf_v_2_1/trainer

In [ ]:
%%writefile flowersmodel_augment_tf_v_2_1/trainer/__init__.py
##

In [ ]:
%%writefile flowersmodel_augment_tf_v_2_1/trainer/task.py
import argparse
import json
import os

from . import model
import tensorflow as tf

if __name__=='__main__':
    parser = argparse.ArgumentParser()

    # Input Arguments
    parser.add_argument(
        '--batch_size',
        help="Batch size for training steps.",
        type=int,
        default=100
    )

    parser.add_argument(
        '--learning_rate',
        help="Initial learning rate for training.",
        type=float,
        default=0.01
    )

    parser.add_argument(
        '--train_steps',
        help="Steps to run the train jobs for. A step is one batch size.",
        type=int,
        default=100
    )

    parser.add_argument(
        '--output_dir',
        help="GCS location to write checkpoints and export model",
        required=True
    )

    parser.add_argument(
        '--train_data_path',
        help="Location of train file, which contain training image URL with appropriate label.",
        default="gs://cloud-ml-data/img/flower_photos/train_set.csv"
    )

    parser.add_argument(
        '--eval_data_path',
        help="Location of eval file, which contain evaluation image URL with appropriate label.",
        default="gs://cloud-ml-data/img/flower_photos/eval_set.csv"
    )

    # Build list model_fn's for help message.
    model_names = [name.replace("_model", "") for name in dir(model) if name.endswith('_model')]

    parser.add_argument(
        '--model',
        help="Type of model. Supported types are {}".format(model_names),
        required=True
    )

    parser.add_argument(
        '--job-dir',
        help="This model ignore this field, but it is required by gcloud.",
        default='junk'
    )

    parser.add_argument(
        '--augment',
        help="If specified augment image data.",
        dest='augment',
        action='store_true'
    )

    parser.set_defaults(augment=False)

    # Optional hyper parameter used by cnn.
    parser.add_argument(
        '--ksize1',
        help="Kernel size of the first layer of the cnn.",
        type=int,
        default=5
    )

    parser.add_argument(
        '--ksize2',
        help="Kernel size of the second layer of the cnn.",
        type=int,
        default=5
    )

    parser.add_argument(
        '--nfil1',
        help="Number of filters in the first layer of the cnn.",
        type=int,
        default=10
    )

    parser.add_argument(
        '--nfil2',
        help="Number of filters in the second layer of the cnn.",
        type=int,
        default=20
    )

    parser.add_argument(
        '--dprob',
        help="Dropout probability for cnn",
        type=float,
        default=0.25
    )

    parser.add_argument(
        '--batch_norm',
        help="If specified do batch norm for CNN",
        dest="batch_norm",
        action="store_true"
    )

    parser.set_defaults(batch_norm = False)

    args= parser.parse_args()
    hparams = args.__dict__

    output_dir = hparams.pop('output_dir')

    # Appends trail id to path for hyper parameter tunning.
    output_dir = os.path.join(
        output_dir,
        json.loads(
            os.environ.get("TF_CONFIG", "{}")
        ).get("task", {}).get("trail", "")
    )

    # Run the training job.
    model.train_and_evaluate(output_dir, hparams)

In [ ]:
%%writefile flowersmodel_augment_tf_v_2_1/trainer/model.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(
    tf.compat.v1.logging.INFO
)

LIST_OF_LABELS = 'daisy,dandelion,roses,sunflowers,tulips'.split(',')
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CHANNELS = 3
NCLASSES = 5

# Build Keras model using Keras Sequential API
def linear_model(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=[IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS],
        name="image"
    ))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    return model

def dnn_model(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=[IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS],
        name="image"
    ))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=300,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dense(
        units=100,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dense(
        units=30,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    return model

def dnn_dropout_model(hparams):
    dprob = hparams.get("dprob", 0.10)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=[IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS],
        name="image"
    ))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=300,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dense(
        units=100,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dense(
        units=30,
        activation=tf.nn.relu
    ))
    model.add(tf.keras.layers.Dropout(
        rate=dprob
    ))
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    return model

def cnn_model(hparams):
    ksize1 = hparams.get("ksize1", 5)
    ksize2 = hparams.get("ksize2", 5)
    nfil1 = hparams.get("nfil1", 10)
    nfil2 = hparams.get("nfil2", 20)
    dprob = hparams.get("dprob", 0.25)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=[IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS],
        name="image"
    )) # Shape = (?, 224, 224, 3)
    model.add(tf.keras.layers.Conv2D(
        filters=nfil1,
        kernel_size=ksize1,
        padding='same',
        activation=tf.nn.relu
    )) # Shape = (?, 224, 224, nfil1)
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=2,
        strides=2
    )) # Shape = (?, 112, 112, nfil1)
    model.add(tf.keras.layers.Conv2D(
        filters=nfil2,
        kernel_size=ksize2,
        padding='same',
        activation=tf.nn.relu
    )) # Shape = (?, 112, 112, nfil2)
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=2,
        strides=2
    )) # Shape = (?, 56, 56, nfil2)
    model.add(tf.keras.layers.Flatten())
    
    # Apply batch normalization.
    if hparams["batch_norm"]:
        model.add(tf.keras.layers.Dense(
            units=300,
            activation=tf.nn.relu
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation(
            activation=tf.nn.relu
        ))
    else:
        model.add(tf.keras.layers.Dense(
            units=300,
            activation=tf.nn.relu
        ))
    
    # Apply Dropout
    model.add(tf.keras.layers.Dropout(
        rate=dprob
    ))
    
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=None
    ))
    
    # Apply Batch Normalization once more.
    if hparams["batch_norm"]:
        model.add(tf.keras.layers.BatchNormalization())
    
    # SoftMax Layer.
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    
    return model

MAX_DELTA = 63.0 / 255.0 # Change brightness by at most 17.7%
CONTRAST_LOWER = 0.2
CONTRAST_UPPER = 1.8

def read_and_preprocess(image_bytes, label=None, augment=False):
    # Convert the compressed string to a 3D uint8 tensor.
    img = tf.image.decode_jpeg(
        contents=image_bytes,
        channels=NUM_CHANNELS
    )
    # Use 'convert_image_dtype' to converts to floats in the 
    # [0,1] range.
    img = tf.image.convert_image_dtype(
        image=img,
        dtype=tf.float32
    )
    # Resize the image to a desired size.
    img = tf.image.resize(
        images=img,
        size=[IMG_HEIGHT + 10, IMG_WIDTH + 10],
    )
    
    if augment:
        img = tf.image.random_crop(
            value=img,
            size=[IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS]
        )
        img = tf.image.random_flip_left_right(
            image=img
        )
        img = tf.image.random_brightness(
            image=img,
            max_delta=MAX_DELTA
        )
        img = tf.image.random_contrast(
            image=img,
            lower=CONTRAST_LOWER,
            upper=CONTRAST_UPPER
        )
    
    return img, label

def read_and_preprocess_with_augment(image_bytes, label=None):
    return read_and_preprocess(
        image_bytes=image_bytes,
        label=label,
        augment=True
    )

# Create Serving input function for inference.
def serving_input_fn():
    feature_placeholders = {
        "image_bytes":tf.compat.v1.placeholder(
            dtype=tf.string,
            shape=[]
        )
    }
    img = read_and_preprocess(
        image_bytes=feature_placeholders["image_bytes"]
    )
    return tf.estimator.export.ServingInputReceiver(
        features=img,
        receiver_tensors=feature_placeholders,
    )

def make_input_fn(csv_of_filenames, batch_size, training=True, augment=False):
    def _input_fn():
        def decode_csv(csv_row):
            record_defaults = ["path", "flowers"]
            filename, label_string = tf.io.decode_csv(
                records=csv_row,
                record_defaults=record_defaults
            )
            image_bytes = tf.io.read_file(
                filename=filename
            )
            label = tf.math.equal(LIST_OF_LABELS, label_string)
            return image_bytes, label
        
        # Create tf.data.dataset from filename
        dataset = tf.data.TextLineDataset(
            filenames=csv_of_filenames
        ).map(decode_csv)
        
        if augment:
            dataset = dataset.map(read_and_preprocess_with_augment)
        else:
            dataset = dataset.map(read_and_preprocess)
            
        if training:
            num_epochs = None # Indefinately
            dataset = dataset.shuffle(
                buffer_size = 10 * batch_size
            ) 
        else:
            num_epochs=1 # Each photo used once
            
        dataset = dataset.repeat(
            count=num_epochs
        ).batch(batch_size=batch_size)
        
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn

# Wrapper function to build selected Keras model type.
def image_classifier(hparams):
    model_functions = {
        "linear": linear_model,
        "dnn": dnn_model,
        "dnn_dropout":dnn_dropout_model,
        "cnn":cnn_model
    }
    
    # Get function pointer for selected model type
    model_function = model_functions[hparams["model"]]
    
    # Build selected Keras model.
    model = model_function(hparams)
    
    return model

# Create train_and_evaluate function.
def train_and_evaluate(output_dir, hparams):
    # Ensure filewriter cache is clear for TensorBoard event file.
    tf.compat.v1.summary.FileWriterCache.clear()
    
    EVAL_INTERVAL = 60
    
    # Build Keras model.
    model = image_classifier(hparams)
    
    # Compile Keras model with Optimizer, Loss Function
    # and eval metric.
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    # Convert keras model to estimator.
    estimator = tf.keras.estimator.model_to_estimator(
        keras_model=model,
        model_dir=output_dir,
        config=tf.estimator.RunConfig(
            save_checkpoints_secs=EVAL_INTERVAL
        )
    )
    
    # Set estimator train_spec
    train_spec = tf.estimator.TrainSpec(
        input_fn=make_input_fn(
            csv_of_filenames=hparams['train_data_path'],
            batch_size=hparams['batch_size'],
            training=True,
            augment=hparams['augment']
        ),
        max_steps=hparams['train_steps']
    )
    
    # Create exporter that use serving_input_fn() to
    # create saved_model for serving.
    exporter = tf.estimator.LatestExporter(
        name="exporter",
        serving_input_receiver_fn=serving_input_fn,
    )
    
    # Set estimators eval_spec.
    eval_spec = tf.estimator.EvalSpec(
        input_fn=make_input_fn(
            csv_of_filenames=hparams['eval_data_path'],
            batch_size=hparams['batch_size'],
            training=False,
        ),
        steps=None,
        exporters=exporter,
        throttle_secs=EVAL_INTERVAL
    )
    
    # Run train_and_evaluate
    tf.estimator.train_and_evaluate(
        estimator=estimator, 
        train_spec=train_spec, 
        eval_spec=eval_spec
    )

## Run as a Python module

Let's first run it locally for a short while to test the code works. Note the --model parameter

In [ ]:
%%bash
rm -rf flowersmodel.tar.gz flowers_trained
gcloud ml-engine local train \
    --module-name=flowersmodel_augment_tf_v_2_1.task \
    --package-path=./flowersmodel_augment_tf_v_2_1 \
    -- \
    --output_dir=./flowers_trained \
    --train_steps=5\
    --learning_rate=0.01 \
    --batch_size=2 \
    --model=$MODEL_TYPE \
    --augment=True \
    --train_data_path=gs://cloud-ml-data/img/flowers_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flowers_photos/eval_set.csv

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u + %y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=flowersmodel_augment_tf_v_2_1.task \
    --package-path=./flowersmodel_augment_tf_v_2_1 \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=1000 \
    --learning_rate=0.01 \
    --batch_size=40 \
    --model=$MODEL_TYPE \
    --augment=True \
    --batch_norm=True \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

## Deploying and predicting with model

Deploy the model:

In [ ]:
%%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ...this will take a few minutes"
#gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions ${REGION}
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

The online prediction service expects images to be base64 encoded as described [here](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#binary_data_in_prediction_input).

In [ ]:
%%bash
IMAGE_URL=gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg

# Copy the image to local disk.
gsutil cp $IMAGE_URL flower.jpg

# Base64 encode and create request message in json format.
python -c 'import base64, sys, json; img = base64.b64encode(open("flower.jpg", "rb").read()).decode(); print(json.dumps({"image_bytes":{"b64": img}}))' &> request.json

Send it to the prediction service

In [ ]:
%%bash
gcloud ml-engine predict \
    --model=flowers \
    --version=${MODEL_TYPE} \
    --json-instances=./request.json